# Improving proofreading interface
1. Render skeleton points
2. Make largest fragment red
2. Make all others a color range based on size

In [1]:
import sys
sys.path.append('/Users/brandon/Documents/Repositories/Python/')

In [2]:
from FANC_auto_recon.segmentation import authentication_utils,rootID_lookup
from FANC_auto_recon.proofreading import proofreading_utils
from FANC_auto_recon.connectomics_bot import response_methods
from FANC_auto_recon.skeletonization import catmaid_utilities
import pymaid
from cloudvolume import CloudVolume
from nglui.statebuilder import *
import numpy as np
import pandas as pd
from FANC_auto_recon.transforms import realignment
import json
from matplotlib import cm
from matplotlib import colors

In [28]:
def skel2seg(skid, project=13, segment_threshold = 10, node_threshold = None):
    
    CI = catmaid_utilities.catmaid_login('FANC',project)
    try:
        n = pymaid.get_neurons(skid)
    except:
        return('No matching skeleton ID in project {}'.format(project))
    
    n.downsample(inplace=True)

    nodes = n.nodes[['x','y','z']].values/ np.array([4.3,4.3,45])
    skeleton_nodes_voxel = realignment.fanc3_to_4(nodes, verbose=False)

    target_volume = CloudVolume(authentication_utils.get_cv_path('FANC_production_segmentation')['url'], use_https=True, agglomerate=False)
    transformed = realignment.fanc3_to_4(skeleton_nodes_voxel)

    seg_ids = rootID_lookup.segIDs_from_pts(target_volume,transformed)
    
    neuron_df,skeleton_df = fragment_dataframes(seg_ids,
                                     transformed,
                                     segment_threshold=segment_threshold,
                                     node_threshold=node_threshold)
    annotations = [{'name':'skeleton coords',
                'type':'points',
                'data': skeleton_df}]
    return render_scene(neurons=neuron_df,annotations=annotations,return_as='url')
 
    
def fragment_dataframes(seg_ids,coords,segment_threshold=None,node_threshold=20):

    ids,counts = np.unique(seg_ids,return_counts=True)
    value_counts = np.array(list(zip(ids,counts)),dtype=int)
    value_counts=value_counts[value_counts[:,0]!=0,:]

    primary_neuron = value_counts[value_counts[:,1]==max(value_counts[:,1]),0]
    fragments = value_counts[value_counts[:,0]!=primary_neuron,:]

    if segment_threshold and not node_threshold:
        ids_to_use = fragments[np.argsort(-fragments[:,1])[0:segment_threshold],0]
    elif node_threshold and not segment_threshold:
        ids_to_use = fragments[fragments[:,1]>node_threshold][:,0]
    elif node_threshold and segment_threshold:
        print('Warning: cannot use segment and node threshold concurrently,defaulting to segment threshold')
        ids_to_use = fragments[np.argsort(-fragments[:,1])]
    else:
        ids_to_use = seg_ids
        
        
        

    skeleton_df = pd.DataFrame(columns=['segment_id','xyz'])
    skeleton_df.xyz = [i for i in coords]
    cmap = cm.get_cmap('viridis', len(ids_to_use)) 
    sk_colors = [colors.rgb2hex(cmap(i)) for i in range(cmap.N)]
    
    
    neuron_df = pd.DataFrame(columns=['segment_id','xyz','color'])
    neuron_df.segment_id = ids_to_use
    
    j = 0
    for i in ids_to_use:
        idx = seg_ids == i
        neuron_df.loc[neuron_df.segment_id ==i,'color'] = sk_colors[j]
        j+=1

    neuron_df = neuron_df.append({'segment_id':primary_neuron[0],'xyz':None,'color':"#ff0000"},ignore_index=True)
    return neuron_df,skeleton_df

def render_scene(neurons = None,
                 img_source = None,
                 seg_source = None,
                 state_server=None,
                 annotations = None,
                 client=None,
                 return_as = 'url'):
                
    
    if client is None:
        client,token = authentication_utils.get_client()


    if neurons is None:
        neurons = pd.DataFrame(columns=['segment_id','xyz','color'])
    elif isinstance(neurons,list):
        
        cmap = cm.get_cmap('Set1', len(neurons)) 
        neurons_df = pd.DataFrame(columns=['segment_id','xyz','color'])
        neurons_df['segment_id'] = neurons
        neurons_df['color'] = [colors.rgb2hex(cmap(i)) for i in range(cmap.N)]

    

    paths = authentication_utils.get_cv_path()

    if img_source is None:
        img_source = paths['Image']['url']
    if seg_source is None:
        seg_source = paths['FANC_production_segmentation']['url']
    if state_server is None:
        state_server = paths['json_server']['url']




    # Set layer segmentation layer
    img_layer = ImageLayerConfig(img_source,name='FANCv4')


    seg_layer = SegmentationLayerConfig(name = 'seg_Mar2021_proofreading',
                                   source = seg_source,
                                   selected_ids_column='segment_id',
                                   color_column= 'color',
                                   fixed_ids= None,
                                   active = False)


    standard_state = StateBuilder(layers=[img_layer,seg_layer],resolution=[4.3,4.3,45])



    # Data Layer(s)

    annotation_states = []
    annotation_data = []
    if annotations is not None:
        for i in annotations:

            if i['type'] is 'points':
                anno_mapper = PointMapper(point_column='xyz')  
   
            anno_layer =  AnnotationLayerConfig(name=i['name'],mapping_rules=anno_mapper)

            annotation_states.append(StateBuilder(layers=[anno_layer],resolution=[4.3,4.3,45]))
            annotation_data.append(i['data'])





    chained_sb = ChainedStateBuilder([standard_state]+annotation_states)
    state = json.loads(chained_sb.render_state([neurons]+annotation_data, return_as='json'))

    # Add brain regions
    state['layers'].append( {"type": "segmentation",
          "mesh": paths['volumes']['url'],
          "objectAlpha": 0.1,
          "hideSegmentZero": False,
          "ignoreSegmentInteractions": True,
          "segmentColors": {
            "1": "#bfbfbf",
            "2": "#d343d6"
          },
          "segments": [
            "1",
            "2"
          ],
          "skeletonRendering": {
            "mode2d": "lines_and_points",
            "mode3d": "lines"
          },
          "name": "volume outlines"
        })

    if return_as is 'url':
        return(paths['neuroglancer_base']['url'] + '?json_url={path}{state_id}'.format(path=paths['json_server']['url'],
                                                                                       state_id=client.state.upload_state_json(state)))
    else:
        return(state)
    


In [30]:
skid = 694501
project = 13

skel2seg(skid,project)


INFO  : Global CATMAID instance set. Caching is ON. (pymaid)
Segmentation IDs: 100%|██████████| 768/768 [01:52<00:00,  6.80it/s]


'https://neuromancer-seung-import.appspot.com/?json_url=https://api.zetta.ai/json/601987990957767144'